In [ ]:
%pip install pandas

In [187]:
import os
import pandas as pd

# Define the highlighted part of the path as a separate variable
root_path = r"C:\datapipes\data"
csv_files = ['facilities.csv', 'collaterals.csv', 'counterparties.csv']
csv_dfs = [file_name.replace('.csv', '') for file_name in csv_files]

# Initialize an empty dictionary to store the DataFrames
dataframes = {}

# Iterate through the CSV files
for file in csv_files:
    # Remove the ".csv" extension from the file name
    file_name = os.path.splitext(file)[0]
    file_path = os.path.join(root_path, file)
    # Check if the file exists before reading it
    if os.path.exists(file_path):
        # Read the CSV file into a DataFrame
        dataframes[file_name] = pd.read_csv(file_path)
    else:
        print(f"File '{file}' does not exist in the specified path.")

# Read the metadata.csv into a DataFrame
metadata_df = pd.read_csv(os.path.join(root_path, 'metadata.csv'))
metadata_df.head()

,table,fieldname,label,type
0,collaterals,collateral_id,Collateral Identification Number,int8
1,collaterals,type,Collateral Type,category
2,collaterals,orig_value,Original Value of Collateral,float
3,corporations_data,corp_name,Corporation Name,str
4,corporations_data,zip_code,Zip Code,category


In [190]:
# List of unique table names within the metadata
unique_tables = list(metadata_df['table'].unique())

# Check if all unique_tables values are found in csv_files
if not set(csv_dfs).issubset(unique_tables):
    missing_tables = set(csv_dfs) - set(unique_tables)
    print(f"The following unique_tables values are not found in metadata_df table values: {missing_tables}")
else:
    print("All csv_files values are found in unique_tables.", "\n")

    dataframes_clean = {}
    dataframes_clean_nocat = {}

    for table_name in csv_dfs:
        print("DATAFRAME in the iteration: ", table_name, "\n")
        subset_metadata_df = metadata_df[metadata_df['table'] == table_name]

        # Convert the DataFrame to a dictionary {field_name: data_type}
        metadata_dict = pd.Series(subset_metadata_df.type.values, index=subset_metadata_df.fieldname).to_dict()

        print("The metadata dictionary with columns and data types")
        print(metadata_dict, "\n")

        table = dataframes[table_name]
        table_nocat = dataframes[table_name].copy()

        # Convert each column to the specified data type - categorical types are not included
        for i, (fieldname, dtype) in enumerate(metadata_dict.items()):
            if dtype == 'datetime64':
                table_nocat[fieldname] = pd.to_datetime(table_nocat[fieldname])
            elif dtype == 'category':
                table_nocat[fieldname] = table_nocat[fieldname].astype('object')
            else:
                table_nocat[fieldname] = table_nocat[fieldname].astype(dtype)
        # Stack dataframes into a dictionary
        dataframes_clean_nocat[table_name] = table_nocat

        # Convert each column to the specified data type - including categorical types
        for fieldname, dtype in metadata_dict.items():
            if dtype == 'datetime64':
                table[fieldname] = pd.to_datetime(table[fieldname])
            else:
                table[fieldname] = table[fieldname].astype(dtype)
        
        # Stack dataframes into a dictionary
        dataframes_clean[table_name] = table

        # Now the dataframe has columns types as specified in metadata_dict
        print("Column types without category type >>")
        print(dataframes_clean_nocat[table_name].dtypes, "\n")
        print("Column types with category type >>")
        print(dataframes_clean[table_name].dtypes, "\n")

        # Calculate the memory usage of the DataFrame
        memory_usage_nocat = dataframes_clean_nocat[table_name].memory_usage(deep=True).sum()
        memory_usage = dataframes_clean[table_name].memory_usage(deep=True).sum()
        # Print the memory usage in bytes
        print("Memory Usage without category type:", memory_usage_nocat, "bytes")
        print("Memory Usage with category type:", memory_usage, "bytes", "\n")


All csv_files values are found in unique_tables. 

DATAFRAME in the iteration:  facilities 

The metadata dictionary with columns and data types
{'facility_id': 'int8', 'counterparty_id': 'int8', 'start_date': 'datetime64', 'end_date': 'datetime64', 'facility_type': 'category', 'amount': 'float'} 

Column types without category type >>
facility_id                  int8
counterparty_id              int8
start_date         datetime64[ns]
end_date           datetime64[ns]
facility_type              object
amount                    float64
dtype: object 

Column types with category type >>
facility_id                  int8
counterparty_id              int8
start_date         datetime64[ns]
end_date           datetime64[ns]
facility_type            category
amount                    float64
dtype: object 

Memory Usage without category type: 92895 bytes
Memory Usage with category type: 27576 bytes 

DATAFRAME in the iteration:  collaterals 

The metadata dictionary with columns and data typ

In [189]:
facilities = dataframes_clean['facilities']
facilities["facility_type"] = facilities["facility_type"].astype('object')
facilities.dtypes

facility_id                  int8
counterparty_id              int8
start_date         datetime64[ns]
end_date           datetime64[ns]
facility_type              object
amount                    float64
dtype: object